<a href="https://colab.research.google.com/github/cwinsor/medical_image_uw_madison/blob/main/colab_01_data_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preprocessing
This notebook runs data_preprocess.py

In [19]:
import sys
import os
from shutil import rmtree

import numpy as np
import pandas as pd
import cv2
from PIL import Image
import glob
from tqdm.auto import tqdm
import albumentations as A

In [21]:
git_url = 'https://github.com/cwinsor/medical_image_uw_madison.git'
git_folder = '/content/medical_image_uw_madison'

# populate the git and it's submodule
if 'google.colab' in sys.modules:
    print("host is colab")
    if os.path.isdir(git_folder):
        print('update existing ' + git_folder)
        rmtree(git_folder)
    else:
        print('populate ' + git_folder)
    !git clone --quiet $git_url
    os.chdir(git_folder)
    # !ls -la
    !git submodule update --init
    os.chdir('/content/')
    sys.path.append(git_folder)
else:
    print("host is traditional server")
    assert False, "not coded"

host is colab
update existing /content/medical_image_uw_madison
Submodule 'project/Kaggle-UWMGIT' (https://github.com/CarnoZhao/Kaggle-UWMGIT) registered for path 'project/Kaggle-UWMGIT'
Cloning into '/content/medical_image_uw_madison/project/Kaggle-UWMGIT'...
Submodule path 'project/Kaggle-UWMGIT': checked out '91bc93fec8d25bed200a7ae7963c9738ac798353'


In [39]:
os.chdir(git_folder + '//project//Kaggle-UWMGIT')
!pwd

/content/medical_image_uw_madison/project/Kaggle-UWMGIT


In [40]:
def rle_decode(mask_rle, shape):
    s = np.array(mask_rle.split(), dtype=int)
    starts, lengths = s[0::2] - 1, s[1::2]
    ends = starts + lengths
    h, w = shape
    img = np.zeros((h * w,), dtype = np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = 1
    return img.reshape(shape)

def rle_encode(img):
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [41]:
os.chdir("./data/tract")
!pwd

/content/medical_image_uw_madison/project/Kaggle-UWMGIT/data/tract


In [42]:
df_train = pd.read_csv("./train.csv")

FileNotFoundError: ignored

In [25]:
df_train = pd.read_csv("./train.csv")
df_train = df_train.sort_values(["id", "class"]).reset_index(drop = True)
df_train["patient"] = df_train.id.apply(lambda x: x.split("_")[0])
df_train["days"] = df_train.id.apply(lambda x: "_".join(x.split("_")[:2]))
num_slices = len(np.unique(df_train.id))
num_empty_slices = df_train.groupby("id").apply(lambda x: x.segmentation.isna().all()).sum()
num_patients = len(np.unique(df_train.patient))
num_days = len(np.unique(df_train.days))
print({
    "#slices:": num_slices,
    "#empty slices:": num_empty_slices,
    "#patients": num_patients,
    "#days": num_days
})

FileNotFoundError: ignored